In [2]:
from bs4 import BeautifulSoup as bs
import requests
import re
import pandas as pd

In [4]:
url_inicial = 'https://www.imovirtual.com/comprar/apartamento/seixal-arrentela-e-aldeia-de-paio-pires-seixal/?search%5Bfilter_float_m%3Afrom%5D=90&search%5Bfilter_float_m%3Ato%5D=120&search%5Bfilter_enum_rooms_num%5D%5B0%5D=3&search%5Bregion_id%5D=15&search%5Bsubregion_id%5D=227&search%5Bcity_id%5D=15738887&page=1'
r = requests.get(url_inicial)

soup = bs(r.content)

contents = soup.prettify()

In [5]:
# Obter o número de páginas dos resultados
page_count = re.findall(r'page_count=\d', contents)[0].split("=")
page_count = int(page_count[1])
page_count

4

In [6]:
# Obter o link com os detalhes de cada casa
pages_html = []
for i in range(1, page_count + 1):
    url_page = f'https://www.imovirtual.com/comprar/apartamento/seixal-arrentela-e-aldeia-de-paio-pires-seixal/?search%5Bfilter_float_m%3Afrom%5D=90&search%5Bfilter_float_m%3Ato%5D=120&search%5Bfilter_enum_rooms_num%5D%5B0%5D=3&search%5Bregion_id%5D=15&search%5Bsubregion_id%5D=227&search%5Bcity_id%5D=15738887&page={i}'
    r = requests.get(url_page)
    soup = bs(r.content)
    pages_html.append(soup.find_all(class_='offer-item-details'))

In [7]:
# Regex para conseguir extrair o url com os detalhes de cada casa
casa_url = re.findall(r'https[^\"]+', str(pages_html))

In [9]:
titles = []
links = []
prices = []
locations = []
prices_m2 = []
properties_titles = []
properties_values = []
descriptions = []
carateristicas = []
i=0
for casa in casa_url:
    r = requests.get(casa)
    soup = bs(r.content, 'html.parser')
    print(i)
    print(casa)
    # Titulos dos campos das Propriedades
    properties_title = soup.find(class_='css-1sxg93g e1t9fvcw3')
    properties_title_total = properties_title.find_all(class_='css-o4i8bk ev4i3ak2')
    for title in properties_title_total:
        # Titulo do Anúncio 
        titles.append(soup.find_all(class_='css-11kn46p eu6swcv17')[0].text)
        # Link do Anúncio
        links.append(casa)
        # Preço
        prices.append(soup.find_all(class_='css-1sxg93g e1t9fvcw3')[0].find(class_='css-8qi9av eu6swcv16').text)
        # Localização
        locations.append(soup.find_all(class_='css-1sxg93g e1t9fvcw3')[0].find(class_='css-1k12nzr eu6swcv12').text)
        # Preço por m2
        prices_m2.append(soup.find_all(class_='css-1sxg93g e1t9fvcw3')[0].find(class_='css-1p44dor eu6swcv13').text)
        properties_titles.append(title.text)
        # Descrição do anúncio
        description = soup.find(class_='css-1vfwbw8 e1r1048u3')
        info_description = description.find_all(["p", "li"])
        info_description_list = []
        if len(info_description) == 0:
            info_description = soup.find(class_='css-1vfwbw8 e1r1048u3').text
            info_description_list.append(info_description)
        else:
            for item in info_description:
                info_description_list.append(item.text)
        descriptions.append(info_description_list)
        try:
            details = soup.find(class_='css-belab5 eekkszi1')
            info_details = details.find_all("li")
            info_details_list = []
            for item in info_details:
                info_details_list.append(item.text)
            carateristicas.append(info_details_list)
        except AttributeError as e:
            carateristicas.append("N/A")
    properties = soup.find(class_='css-1sxg93g e1t9fvcw3')
    properties_total = properties.find_all(class_='css-1ytkscc ev4i3ak0')
    for property in properties_total:
        properties_values.append(property.text)
    i+=1

0
https://www.imovirtual.com/pt/anuncio/apartamento-de-4-assoalhas-totalmente-remodeladas-em-paio-pires-ID18tNp.html#f1eb8021bf
1
https://www.imovirtual.com/pt/anuncio/apartamento-t3-de-103-m2-totalmente-renovado-2-andar-ca-ID18xUK.html#f1eb8021bf
2
https://www.imovirtual.com/pt/anuncio/luminoso-t3-garagem-box-arrecadacao-varanda-larga-torre-da-marinha-ID18egH.html#f1eb8021bf
3
https://www.imovirtual.com/pt/anuncio/apartamento-t3-na-torre-da-marinha-com-garagem-para-2-viaturas-ID18pHw.html#f1eb8021bf
4
https://www.imovirtual.com/pt/anuncio/t3-com-otimas-areas-na-arrentela-t3-with-great-areas-i-ID18cD3.html#f1eb8021bf
5
https://www.imovirtual.com/pt/anuncio/apartamento-t3-com-arrecadacao-e-parqueamento-torre-da-ID183D9.html#f1eb8021bf
6
https://www.imovirtual.com/pt/anuncio/t3-colinas-do-sul-ID18kZa.html#f1eb8021bf
7
https://www.imovirtual.com/pt/anuncio/t3-com-parqueamento-e-arrecadacao-urbanizacao-colinas-do-sul-ID17F3d.html#f1eb8021bf
8
https://www.imovirtual.com/pt/anuncio/t3-2-anda

In [10]:
test = pd.DataFrame(data={"Titulo":titles, "Preco":prices, "Preco por m2":prices_m2, "Titulo Propriedades":properties_titles,
                    "Valores Propriedades":properties_values, "Descricao":descriptions, "Caracteristicas":carateristicas, "Link":links, "Localização":locations})
test

,Titulo,Preco,Preco por m2,Titulo Propriedades,Valores Propriedades,Descricao,Caracteristicas,Link,Localização
0,Apartamento de 4 Assoalhas totalmente remodela...,153 000 €,1 594 €/m²,Área útil (m²):,96 m²,[Procura um apartamento de 4 assoalhadas remod...,"[Despensa, Fibra Óptica, Marquise, Termoacumul...",https://www.imovirtual.com/pt/anuncio/apartame...,"Rua Ferreira de Castro, Seixal, Arrentela e Al..."
1,Apartamento de 4 Assoalhas totalmente remodela...,153 000 €,1 594 €/m²,Área bruta (m²):,125 m²,[Procura um apartamento de 4 assoalhadas remod...,"[Despensa, Fibra Óptica, Marquise, Termoacumul...",https://www.imovirtual.com/pt/anuncio/apartame...,"Rua Ferreira de Castro, Seixal, Arrentela e Al..."
2,Apartamento de 4 Assoalhas totalmente remodela...,153 000 €,1 594 €/m²,Tipologia:,T3,[Procura um apartamento de 4 assoalhadas remod...,"[Despensa, Fibra Óptica, Marquise, Termoacumul...",https://www.imovirtual.com/pt/anuncio/apartame...,"Rua Ferreira de Castro, Seixal, Arrentela e Al..."
3,Apartamento de 4 Assoalhas totalmente remodela...,153 000 €,1 594 €/m²,Casas de Banho:,1,[Procura um apartamento de 4 assoalhadas remod...,"[Despensa, Fibra Óptica, Marquise, Termoacumul...",https://www.imovirtual.com/pt/anuncio/apartame...,"Rua Ferreira de Castro, Seixal, Arrentela e Al..."
4,Apartamento de 4 Assoalhas totalmente remodela...,153 000 €,1 594 €/m²,Certificado Energético:,D,[Procura um apartamento de 4 assoalhadas remod...,"[Despensa, Fibra Óptica, Marquise, Termoacumul...",https://www.imovirtual.com/pt/anuncio/apartame...,"Rua Ferreira de Castro, Seixal, Arrentela e Al..."
...,...,...,...,...,...,...,...,...,...
653,Apartamento T3 com garagem Box para 2 Carros,260 000 €,2 167 €/m²,Empreendimento:,não,[DescriçãoApartamento de 4 Assoalhadas com gar...,N/A,https://www.imovirtual.com/pt/anuncio/apartame...,"Seixal, Arrentela e Aldeia de Paio Pires, Seix..."
654,Apartamento T3 com garagem Box para 2 Carros,260 000 €,2 167 €/m²,Tipologia:,T3,[DescriçãoApartamento de 4 Assoalhadas com gar...,N/A,https://www.imovirtual.com/pt/anuncio/apartame...,"Seixal, Arrentela e Aldeia de Paio Pires, Seix..."
655,Apartamento T3 com garagem Box para 2 Carros,260 000 €,2 167 €/m²,Casas de Banho:,2,[DescriçãoApartamento de 4 Assoalhadas com gar...,N/A,https://www.imovirtual.com/pt/anuncio/apartame...,"Seixal, Arrentela e Aldeia de Paio Pires, Seix..."
656,Apartamento T3 com garagem Box para 2 Carros,260 000 €,2 167 €/m²,Certificado Energético:,E,[DescriçãoApartamento de 4 Assoalhadas com gar...,N/A,https://www.imovirtual.com/pt/anuncio/apartame...,"Seixal, Arrentela e Aldeia de Paio Pires, Seix..."


In [11]:
test_pivot = test.pivot(index="Link", columns="Titulo Propriedades", values='Valores Propriedades')

In [12]:
test_v2 = test_pivot.merge(test, on='Link')

In [13]:
test_v3 = test_v2.drop(['Titulo Propriedades', 'Valores Propriedades', 'Descricao'], axis=1)
test_v3

,Link,Ano de construção:,Casas de Banho:,Certificado Energético:,Condição:,Empreendimento:,Tipologia:,Área bruta (m²):,Área útil (m²):,Titulo,Preco,Preco por m2,Caracteristicas,Localização
0,https://www.imovirtual.com/pt/anuncio/apartame...,1988,2,F,Usado,não,T3,NaN,100 m²,Apartamento - 100 m2 - T3,165 000 €,1 650 €/m²,N/A,"Seixal, Arrentela e Aldeia de Paio Pires, Seix..."
1,https://www.imovirtual.com/pt/anuncio/apartame...,1988,2,F,Usado,não,T3,NaN,100 m²,Apartamento - 100 m2 - T3,165 000 €,1 650 €/m²,N/A,"Seixal, Arrentela e Aldeia de Paio Pires, Seix..."
2,https://www.imovirtual.com/pt/anuncio/apartame...,1988,2,F,Usado,não,T3,NaN,100 m²,Apartamento - 100 m2 - T3,165 000 €,1 650 €/m²,N/A,"Seixal, Arrentela e Aldeia de Paio Pires, Seix..."
3,https://www.imovirtual.com/pt/anuncio/apartame...,1988,2,F,Usado,não,T3,NaN,100 m²,Apartamento - 100 m2 - T3,165 000 €,1 650 €/m²,N/A,"Seixal, Arrentela e Aldeia de Paio Pires, Seix..."
4,https://www.imovirtual.com/pt/anuncio/apartame...,1988,2,F,Usado,não,T3,NaN,100 m²,Apartamento - 100 m2 - T3,165 000 €,1 650 €/m²,N/A,"Seixal, Arrentela e Aldeia de Paio Pires, Seix..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
653,https://www.imovirtual.com/pt/anuncio/vende-se...,2010,2,C,NaN,não,T3,154 m²,116 m²,Vende-se imóvel de 4 assoalhadas .,275 000 €,2 371 €/m²,"[Armário, Porta Blindada, Cozinha Equipada, Es...","Rua Florbela Espanca, Seixal, Arrentela e Alde..."
654,https://www.imovirtual.com/pt/anuncio/vende-se...,2010,2,C,NaN,não,T3,154 m²,116 m²,Vende-se imóvel de 4 assoalhadas .,275 000 €,2 371 €/m²,"[Armário, Porta Blindada, Cozinha Equipada, Es...","Rua Florbela Espanca, Seixal, Arrentela e Alde..."
655,https://www.imovirtual.com/pt/anuncio/vende-se...,2010,2,C,NaN,não,T3,154 m²,116 m²,Vende-se imóvel de 4 assoalhadas .,275 000 €,2 371 €/m²,"[Armário, Porta Blindada, Cozinha Equipada, Es...","Rua Florbela Espanca, Seixal, Arrentela e Alde..."
656,https://www.imovirtual.com/pt/anuncio/vende-se...,2010,2,C,NaN,não,T3,154 m²,116 m²,Vende-se imóvel de 4 assoalhadas .,275 000 €,2 371 €/m²,"[Armário, Porta Blindada, Cozinha Equipada, Es...","Rua Florbela Espanca, Seixal, Arrentela e Alde..."


In [14]:
final_df = test_v3.drop_duplicates(subset=['Link'], keep='first')
final_df

,Link,Ano de construção:,Casas de Banho:,Certificado Energético:,Condição:,Empreendimento:,Tipologia:,Área bruta (m²):,Área útil (m²):,Titulo,Preco,Preco por m2,Caracteristicas,Localização
0,https://www.imovirtual.com/pt/anuncio/apartame...,1988,2,F,Usado,não,T3,NaN,100 m²,Apartamento - 100 m2 - T3,165 000 €,1 650 €/m²,N/A,"Seixal, Arrentela e Aldeia de Paio Pires, Seix..."
7,https://www.imovirtual.com/pt/anuncio/apartame...,1987,2,D,NaN,não,T3,NaN,98 m²,Apartamento - 98 m2 - T3,150 000 €,1 531 €/m²,N/A,"Seixal, Arrentela e Aldeia de Paio Pires, Seix..."
13,https://www.imovirtual.com/pt/anuncio/apartame...,NaN,1,D,Renovado,NaN,T3,125 m²,96 m²,Apartamento de 4 Assoalhas totalmente remodela...,153 000 €,1 594 €/m²,"[Despensa, Fibra Óptica, Marquise, Termoacumul...","Rua Ferreira de Castro, Seixal, Arrentela e Al..."
19,https://www.imovirtual.com/pt/anuncio/apartame...,1987,2,D,Usado,não,T3,97 m²,97 m²,Apartamento localizado na Arrentela junto ao rio,175 000 €,1 804 €/m²,[Negociável],"Seixal, Arrentela e Aldeia de Paio Pires, Seix..."
27,https://www.imovirtual.com/pt/anuncio/apartame...,2022,2,A,Em construção,sim,T3,130 m²,110 m²,Apartamento T3,240 000 €,2 182 €/m²,"[Ar Condicionado, Armário, Arrecadação, Aspira...","Seixal, Arrentela e Aldeia de Paio Pires, Seix..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
623,https://www.imovirtual.com/pt/anuncio/t3-na-ar...,NaN,2,D,Usado,não,T3,96 m²,"98,73 m²",T3 na Arrentela,155 000 €,1 570 €/m²,"[Termoacumulador, Vista de campo/serra, Portar...","Seixal, Arrentela e Aldeia de Paio Pires, Seix..."
630,https://www.imovirtual.com/pt/anuncio/t3-paio-...,1984,2,D,Usado,não,T3,"91,86 m²","91,86 m²",T3 Paio Pires,133 000 €,1 448 €/m²,[Ginásio],"Seixal, Arrentela e Aldeia de Paio Pires, Seix..."
638,https://www.imovirtual.com/pt/anuncio/t3-para-...,NaN,2,B,Usado,não,T3,95 m²,95 m²,"T3 para venda c/ 2 wc´s, Arrentela",136 000 €,1 432 €/m²,"[Despensa, Marquise]","Seixal, Arrentela e Aldeia de Paio Pires, Seix..."
645,https://www.imovirtual.com/pt/anuncio/t3-st-ma...,NaN,NaN,A,Em construção,não,T3,150 m²,"90,50 m²",T3 Stª Marta,250 000 €,2 762 €/m²,N/A,"Seixal, Arrentela e Aldeia de Paio Pires, Seix..."


In [15]:
final_df.dtypes

Link                       object
Ano de construção:         object
Casas de Banho:            object
Certificado Energético:    object
Condição:                  object
Empreendimento:            object
Tipologia:                 object
Área bruta (m²):           object
Área útil (m²):            object
Titulo                     object
Preco                      object
Preco por m2               object
Caracteristicas            object
Localização                object
dtype: object

In [16]:
final_df['Ano de construção:'] = pd.to_numeric(final_df['Ano de construção:'])

C:\Users\u10054206\AppData\Local\Temp\ipykernel_2204\463094499.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['Ano de construção:'] = pd.to_numeric(final_df['Ano de construção:'])


In [17]:
final_df['Casas de Banho:'] = pd.to_numeric(final_df['Casas de Banho:'])

C:\Users\u10054206\AppData\Local\Temp\ipykernel_2204\1048263096.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['Casas de Banho:'] = pd.to_numeric(final_df['Casas de Banho:'])


In [18]:
final_df['Preco'] = final_df['Preco'].str.replace('€', '', regex=True)
final_df['Preco'] = final_df['Preco'].str.replace(' ', '', regex=True)
final_df['Preco'] = pd.to_numeric(final_df['Preco'])

C:\Users\u10054206\AppData\Local\Temp\ipykernel_2204\1123353810.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['Preco'] = final_df['Preco'].str.replace('€', '', regex=True)
C:\Users\u10054206\AppData\Local\Temp\ipykernel_2204\1123353810.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['Preco'] = final_df['Preco'].str.replace(' ', '', regex=True)
C:\Users\u10054206\AppData\Local\Temp\ipykernel_2204\1123353810.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a 

In [19]:
final_df['Preco por m2'] = final_df['Preco por m2'].str.replace('€/m²', '', regex=True)
final_df['Preco por m2'] = final_df['Preco por m2'].str.replace(' ', '', regex=True)
final_df['Preco por m2'] = pd.to_numeric(final_df['Preco por m2'])

C:\Users\u10054206\AppData\Local\Temp\ipykernel_2204\602223015.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['Preco por m2'] = final_df['Preco por m2'].str.replace('€/m²', '', regex=True)
C:\Users\u10054206\AppData\Local\Temp\ipykernel_2204\602223015.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['Preco por m2'] = final_df['Preco por m2'].str.replace(' ', '', regex=True)
C:\Users\u10054206\AppData\Local\Temp\ipykernel_2204\602223015.py:3: SettingWithCopyWarning: 
A value is tryi

In [20]:
final_df['Área útil (m²):'] = final_df['Área útil (m²):'].str.replace('m²', '', regex=True)
final_df['Área útil (m²):'] = final_df['Área útil (m²):'].str.replace(' ', '', regex=True)
final_df['Área bruta (m²):'] = final_df['Área bruta (m²):'].str.replace('m²', '', regex=True)
final_df['Área bruta (m²):'] = final_df['Área bruta (m²):'].str.replace(' ', '', regex=True)
final_df['Área bruta (m²):'] = final_df['Área bruta (m²):'].str.replace(',', '.', regex=True)
final_df['Área útil (m²):'] = final_df['Área útil (m²):'].str.replace(',', '.', regex=True)

C:\Users\u10054206\AppData\Local\Temp\ipykernel_2204\2651937122.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['Área útil (m²):'] = final_df['Área útil (m²):'].str.replace('m²', '', regex=True)
C:\Users\u10054206\AppData\Local\Temp\ipykernel_2204\2651937122.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['Área útil (m²):'] = final_df['Área útil (m²):'].str.replace(' ', '', regex=True)
C:\Users\u10054206\AppData\Local\Temp\ipykernel_2204\2651937122.py:3: SettingWithCopyWarning: 
A 

In [21]:
final_df['Área útil (m²):'] = pd.to_numeric(final_df['Área útil (m²):'])
final_df['Área bruta (m²):'] = pd.to_numeric(final_df['Área bruta (m²):'])

C:\Users\u10054206\AppData\Local\Temp\ipykernel_2204\3711896288.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['Área útil (m²):'] = pd.to_numeric(final_df['Área útil (m²):'])
C:\Users\u10054206\AppData\Local\Temp\ipykernel_2204\3711896288.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['Área bruta (m²):'] = pd.to_numeric(final_df['Área bruta (m²):'])


In [22]:
final_df.to_excel('imovirtual_prices.xlsx', index=False, encoding='utf-8')